In [1]:
import os
import cv2
from shapely.geometry import Polygon
import pickle
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:

with open(r'D:/dataset.pkl', 'rb') as file:

    dataset = pickle.load(file)

In [24]:
print(dataset[1])

{'image_path': 'D:/hackaton/data/train/cameras\\DpR-Csp-uipv-ShV-V1\\017b87c8-b17d-449e-82b7-bb03190a8f77.jpg', 'human_polygon': <POLYGON ((714 196, 843 196, 843 478, 714 478, 714 196))>, 'camera_label': 'DpR-Csp-uipv-ShV-V1'}


In [25]:
images = []
annotations = []
camera_labels = []

In [26]:
for entry in dataset:
    image = cv2.imread(entry['image_path'])
    image = cv2.resize(image, (224, 224))  # Приведение изображений к одному размеру (при необходимости)
    if entry['human_polygon'] is not None:
        annotation = entry['human_polygon'].bounds  # Получаем прямоугольник из объекта Polygon
    else:
        annotation = [0,0,0,0]
    images.append(image)
    annotations.append(annotation)
    camera_labels.append(entry['camera_label'])

In [27]:
camera_label_mapping = {label: idx for idx, label in enumerate(set(camera_labels))}
camera_labels_int = np.array([camera_label_mapping[label] for label in camera_labels])


In [28]:
camera_labels_categorical = tf.keras.utils.to_categorical(camera_labels_int)

In [29]:
train_images, test_images, train_annotations, test_annotations, train_camera_labels, test_camera_labels = train_test_split(
    np.array(images), np.array(annotations), camera_labels_categorical, test_size=0.2, random_state=42
)

In [ ]:
print(images[1])

In [53]:
input_layer = layers.Input(shape=(224, 224, 3), name='conv2d_18_input')
x = layers.Conv2D(32, (3, 3), activation='relu')(input_layer)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
output_bounding_box = layers.Dense(4, name='bounding_box')(x)
output_camera_label = layers.Dense(len(camera_label_mapping), activation='softmax', name='camera_label')(x)

model = models.Model(inputs=input_layer, outputs=[output_bounding_box, output_camera_label])
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss={'bounding_box': 'mean_squared_error', 'camera_label': 'categorical_crossentropy'},
              metrics={'bounding_box': 'accuracy', 'camera_label': 'accuracy'})


In [54]:
model.fit(
    {'conv2d_18_input': train_images},
    {'bounding_box': train_annotations, 'camera_label': train_camera_labels},
    epochs=10,
    validation_data=({'conv2d_18_input': test_images},
                     {'bounding_box': test_annotations, 'camera_label': test_camera_labels})
)

Epoch 1/10
135/135 [==============================] - 188s 1s/step - loss: 174381.1406 - bounding_box_loss: 174253.5938 - camera_label_loss: 127.5720 - bounding_box_accuracy: 0.5947 - camera_label_accuracy: 0.1265 - val_loss: 151140.6094 - val_bounding_box_loss: 151081.8438 - val_camera_label_loss: 58.7676 - val_bounding_box_accuracy: 0.6208 - val_camera_label_accuracy: 0.2138
Epoch 2/10
135/135 [==============================] - 185s 1s/step - loss: 144064.6719 - bounding_box_loss: 144021.1875 - camera_label_loss: 43.4932 - bounding_box_accuracy: 0.6254 - camera_label_accuracy: 0.3064 - val_loss: 129726.9922 - val_bounding_box_loss: 129702.6797 - val_camera_label_loss: 24.2980 - val_bounding_box_accuracy: 0.5883 - val_camera_label_accuracy: 0.3922
Epoch 3/10
135/135 [==============================] - 180s 1s/step - loss: 122728.5312 - bounding_box_loss: 122701.9375 - camera_label_loss: 26.6203 - bounding_box_accuracy: 0.6299 - camera_label_accuracy: 0.4346 - val_loss: 115906.7031 - va

In [ ]:

results = model.evaluate({'conv2d_18_input': test_images}, {'bounding_box': test_annotations, 'camera_label': test_camera_labels})
print("Loss:", results[0])
print("Bounding Box Accuracy:", results[1])
print("Camera Label Accuracy:", results[2])

34/34 [==============================] - 8s 235ms/step - loss: 83893.8750 - bounding_box_loss: 83890.0781 - camera_label_loss: 3.7855 - bounding_box_accuracy: 0.6468 - camera_label_accuracy: 0.8336
Loss: 83893.875
Bounding Box Accuracy: 83890.078125
Camera Label Accuracy: 3.7855193614959717


In [ ]:
model.save('D:Model')

INFO:tensorflow:Assets written to: D:/hackaton/Work/Model\assets


INFO:tensorflow:Assets written to: D:/hackaton/Work/Model\assets


In [ ]:
from tensorflow.keras.models import save_model

save_model(model, 'D:/Work')

INFO:tensorflow:Assets written to: D:/hackaton/Work\assets


INFO:tensorflow:Assets written to: D:/hackaton/Work\assets


In [58]:
from shapely.geometry import Point, Polygon

In [ ]:

def intersect (human_vertices, danger_zone_vertices):
    # Координаты вершин человеческой фигуры


    
    human_polygon = Polygon(human_vertices)
    danger_zone_polygon = Polygon(danger_zone_vertices)

    # Проверяем пересечение многоугольников
    if human_polygon.intersects(danger_zone_polygon):
        # Вычисляем площади многоугольников
        human_area = human_polygon.area
        danger_zone_area = danger_zone_polygon.area

        # Вычисляем процент пересечения от площади человеческой фигуры
        intersection_area = human_polygon.intersection(danger_zone_polygon).area
        intersection_percentage = (intersection_area / human_area) * 100

        # Проверяем условие на пересечение более чем на 15%
        if intersection_percentage > 15:
            print("Человек пересекает опасную зону на: ", intersection_percentage)
            return true
        else:
            print("Человек пересекает опасную зону менее чем на 15%.")
            return false
    else:
        print("Человек не находится в опасной зоне.")
        return false


In [ ]:
import re
coordinats_zones = {}

datapath = "D:/hackaton/data/train/danger_zones"
for zone_file in os.listdir(datapath):
    file_zone = os.path.join(datapath, zone_file)
    
    if zone_file.endswith(".txt"):
        zone_name = os.path.splitext(zone_file)[0]
        if os.path.exists(file_zone):
            with open(file_zone, 'r', encoding='utf-8') as file:  
                data = file.read()
                
                coordinates = [list(map(int, pair.strip('[]').split(','))) for pair in re.findall(r'\[.*?\]', data)]
                name = zone_file.replace('.txt','')
                coordinats_zones[name.replace('danger_','')] = coordinates

# coordinats_zones будет содержать все координаты из всех файлов опасных зон в виде общего списка с подсписками.


In [89]:
print(coordinats_zones)

{'DpR-Csp-uipv-ShV-V1': [[534, 288], [834, 219], [1365, 580], [1124, 806]], 'Pgp-com2-K-1-0-9-36': [[511, 214], [776, 265], [788, 367], [445, 720], [225, 717], [195, 597], [591, 315], [468, 265]], 'Pgp-lpc2-K-0-1-38': [[181, 321], [378, 310], [379, 360], [553, 334], [544, 274], [907, 227], [996, 363], [895, 390], [881, 435], [582, 491], [570, 435], [375, 459], [371, 541], [170, 551]], 'Phl-com3-Shv2-9-K34': [[1335, 640], [1505, 662], [1491, 776], [1290, 752]], 'Php-Angc-K3-1': [[471, 717], [1434, 737], [1460, 894], [1224, 896], [1223, 761], [692, 754], [680, 916], [444, 906]], 'Php-Angc-K3-8': [[1036, 831], [480, 475], [614, 421], [1171, 691]], 'Php-Ctm-K-1-12-56': [[516, 261], [1344, 580], [452, 1078], [84, 352]], 'Php-Ctm-Shv1-2-K3': [[172, 1080], [115, 745], [441, 669], [422, 540], [864, 421], [864, 259], [1363, 151], [1881, 421], [1593, 529], [1824, 723], [1094, 1080]], 'Php-nta4-shv016309-k2-1-7': [[0, 1080], [0, 712], [192, 518], [384, 518], [825, 97], [902, 97], [1132, 367], [11

In [99]:
import cv2
import numpy as np

# Загрузка и предобработка изображения
image_path = 'D:/hackaton/data/train/danger_zones/Pgp-com2-K-1-0-9-36.jpg'
T_i = cv2.imread(image_path)
T_i = cv2.resize(T_i, (224, 224))
T_i = np.expand_dims(T_i, axis=0)  # Добавляем размерность пакета (batch dimension)

# Получение предсказаний
predictions = model.predict({'conv2d_18_input': T_i})
hum = predictions[0]  # Используйте индекс 0, так как это первый элемент в списке
camera_label_predictions = predictions[1]  # Используйте индекс 1 для второго элемента
print(camera_label_predictions)


1/1 [==============================] - 0s 97ms/step
[[0.0000000e+00 1.6265978e-33 0.0000000e+00 5.1284850e-32 0.0000000e+00
  2.1026236e-32 1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  2.4295374e-34 1.3873855e-34]]


In [ ]:

zone = coordinats_zones[camera_label]

print(intersect(hum, zone))
